In [ ]:
%pip install python-dotenv

## 环境变量

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# print(os.environ)

AI_TEMPERATURE=0.0
AI_MAX_TOKENS=2048

## 模型加载

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

def load_azureLLM():
    # chat
    chat = AzureChatOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_CHAT'),
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    # llm
    llm = AzureOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_LLM'),
        model_name="text-davinci-003",
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    
    # embedding = OpenAIEmbeddings(deployment = os.environ.get('DEPLOYMENT_NAME_EMBEDDING'),chunk_size=1)
    
    return llm,chat

llm,chat = load_azureLLM()

## Chain

In [ ]:

from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.prompts import ChatPromptTemplate


person_info = """{
    "gender": "女",
    "age": "20岁",
    "height": "150cm",
    "weight": "50kg"
}"""


fitness_data = """{
    "action": "深蹲",
    "timing": "60秒",
    "count_type": "次数",
    "count_result": "30次"
}"""

### 顺序链

#### 动作评估链

In [ ]:

evaluate_standards = """评估维度：\
1. 力量：指肌肉在运动中产生的最大力量。常见的力量训练包括举重、俯卧撑、深蹲等。通过计次评估。\
2. 速度：指肌肉完成一定距离或动作所需时间的能力。常见的速度训练包括快速小跑、跳绳等。通过计时评估。\
3. 耐力：指肌肉长时间持续工作的能力。常见的耐力训练包括深蹲跳、高抬腿等。通过计次或计时评估。\
4. 灵敏：指肌肉对刺激的快速反应能力。常见的灵敏度训练包括跳绳、跳跃、击剑等。通过计次评估。\
5. 协调：指身体各部位肌肉协调运动的能力，常见的协调性训练包括瑜伽、桑巴、舞蹈等。通过计时评估。\
6. 稳定：指骨骼及肌肉在身体静止或运动状态下维持平衡的能力。常见的稳定性训练包括平板支撑、单脚站立等。通过计时评估。\
评估方法：\
根据动作的计次和计时数据，严格评估动作在每个维度的得分。\
评估得分，最低0分，最高10分。如果无法评估，则为0分。
"""


evaluate_prompt = ChatPromptTemplate.from_template(
    template='您是资深的健身运动专家，请参考用户的个人信息，根据运动评估标准，严格以对用户的运动数据进行评估。\
        运动评估标准是：{standards},\
        用户的个人信息是：{person},\
        用户的运动数据是：{data}'
    )


chain_evaluate = LLMChain(llm=llm, prompt=evaluate_prompt,output_key="evaluate_result")

#### 制定计划链

In [ ]:

plan_prompt = ChatPromptTemplate.from_template(
    template="您是资深的健身运动专家，根据用户的个人信息，以及运动评价结果，以健身计划制定标准为原则，为用户制定个性化的健身计划。\
        用户的个人信息是：{person},\
        用户的运动评价结果是：{evaluate_result}"
    )
chain_plan = LLMChain(llm=llm, prompt=plan_prompt,output_key="plan_reslut")


#### 动作升降阶链

#### 自定义课程链

#### 构建顺序链

In [ ]:

chain = SequentialChain(chains=[chain_evaluate,chain_plan,chain_user],
                        input_variables=["standards","person","data"],
                        output_variables=["user_result","evaluate_result","plan_reslut"],
                        verbose=True)

#### 执行顺序链

In [ ]:

chain({"standards":evaluate_standards,"person":person_info,"data":fitness_data})

### Chain+Conversation

In [ ]:

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
memory.save_context(
    {"input": "你好"},
    {"output": "我是Power，由利它团队创建？出生于2020年。"}
)

In [ ]:
conversation.predict(input="你好，我想调整一下健身计划")

In [85]:
user_prompt = ChatPromptTemplate.from_template(
    template="您需要从用户的信息中，提炼出用户的意图，并输出对应的目标。我们可以支持的目标有：\
        1. 动作评估\
        2. 调整运动计划\
        3. 创建自定义课程\
        \
        用户的需求是：{user_message}"
    )

chain_user = LLMChain(llm=llm, prompt=user_prompt,output_key="user_reslut")

In [ ]:
prompt_infos = [
    {
        "name": "evaluate", 
        "description": "通过严格执行运动评估标准，参考用户的个人身体数据，来评估用户的运动数据。", 
        "prompt_template": evaluate_prompt
    },
    {
        "name": "plan", 
        "description": "根据用户的需求，参照科学严谨健身计划制定标准，调整用户的健身计划。", 
        "prompt_template": plan_prompt
    }
]